# Homework Starter — Stage 04: Data Acquisition and Ingestion
Name: Mihir Subramaniam


## Objectives
- API ingestion with secrets in `.env`
- Scrape a permitted public table
- Validate and save raw data to `data/raw/`

In [8]:
import os, pathlib, datetime as dt
import requests
import pandas as pd
from bs4 import BeautifulSoup
from dotenv import load_dotenv

RAW = pathlib.Path('/Users/murli/Desktop/bootcamp_mihir_subramaniam/homework/homework4/data/raw'); RAW.mkdir(parents=True, exist_ok=True)
load_dotenv(); print('ALPHAVANTAGE_API_KEY loaded?', bool(os.getenv('ALPHAVANTAGE_API_KEY')))

ALPHAVANTAGE_API_KEY loaded? True


## Helpers (use or modify)

In [9]:
def ts():
    return dt.datetime.now().strftime('%Y%m%d-%H%M%S')

def save_csv(df: pd.DataFrame, prefix: str, **meta):
    mid = '_'.join([f"{k}-{v}" for k,v in meta.items()])
    path = RAW / f"{prefix}_{mid}_{ts()}.csv"
    df.to_csv(path, index=False)
    print('Saved', path)
    return path

def validate(df: pd.DataFrame, required):
    missing = [c for c in required if c not in df.columns]
    return {'missing': missing, 'shape': df.shape, 'na_total': int(df.isna().sum().sum())}

## Part 1 — API Pull (Required)
Choose an endpoint (e.g., Alpha Vantage or use `yfinance` fallback).

In [5]:
!pip install yfinance

In [14]:
SYMBOL = 'MSFT'
USE_ALPHA = bool(os.getenv('ALPHAVANTAGE_API_KEY'))
if USE_ALPHA:
    url = 'https://www.alphavantage.co/query'
    params = {'function':'TIME_SERIES_DAILY','symbol':SYMBOL,'outputsize':'compact','apikey':os.getenv('ALPHAVANTAGE_API_KEY')}
    r = requests.get(url, params=params, timeout=30)
    r.raise_for_status()
    js = r.json()
    key = [k for k in js if 'Time Series' in k][0]
    df_api = pd.DataFrame(js[key]).T.reset_index().rename(columns={'index':'date','4. close':'adj_close'})[['date','adj_close']]
    df_api['date'] = pd.to_datetime(df_api['date']); df_api['adj_close'] = pd.to_numeric(df_api['adj_close'])
else:
    import yfinance as yf
    df_api = yf.download(SYMBOL, period='3mo', interval='1d').reset_index()[['Date','Adj Close']]
    df_api.columns = ['date','adj_close']

v_api = validate(df_api, ['date','adj_close']); v_api

{'missing': [], 'shape': (100, 2), 'na_total': 0}

In [15]:
_ = save_csv(df_api.sort_values('date'), prefix='api', source='alpha' if USE_ALPHA else 'yfinance', symbol=SYMBOL)

Saved /Users/murli/Desktop/bootcamp_mihir_subramaniam/homework/homework4/data/raw/api_source-alpha_symbol-MSFT_20250821-165047.csv


## Part 2 — Scrape a Public Table (Required)
Replace `SCRAPE_URL` with a permitted page containing a simple table.

In [ ]:

import re

SCRAPE_URL = "https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_population"
headers = {"User-Agent": "AFE-Homework/1.0", "Accept-Language": "en-US,en;q=0.9"}

def _clean_numeric_string(s: str) -> str:
    return (
        str(s)
        .replace("\u00a0", " ")
        .replace(",", "")
        .replace("%", "")
        .strip()
    )

def _infer_and_coerce_numeric(df_in: pd.DataFrame, min_numeric_share: float = 0.8) -> pd.DataFrame:
    df = df_in.copy()
    for col in df.columns:
        cleaned = (
            df[col]
            .astype(str)
            .str.replace(r"\[[^\]]*\]", "", regex=True)
            .map(_clean_numeric_string)
        )
        as_num = pd.to_numeric(cleaned, errors="coerce")
        non_null = cleaned.notna().sum()
        numeric_like = as_num.notna().sum()
        
        if non_null > 0 and numeric_like / non_null >= min_numeric_share:
            df[col] = as_num
        else:
            
            df[col] = (
                df[col]
                .astype(str)
                .str.replace(r"\s+", " ", regex=True)
                .str.strip()
            )
    return df

try:
    resp = requests.get(SCRAPE_URL, headers=headers, timeout=30)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, "html.parser")

   
    table = soup.select_one("table.wikitable")
    if table is None:
        raise RuntimeError("No 'wikitable' found on the page.")

    
    first_tr = table.find("tr")
    header_cells = first_tr.find_all(["th", "td"])
    header = [re.sub(r"\s+", " ", th.get_text(strip=True)).replace("\u00a0", " ") for th in header_cells]

    
    data = []
    for tr in table.find_all("tr")[1:]:
        tds = tr.find_all("td")
        if not tds:
            continue
        row = [re.sub(r"\s+", " ", td.get_text(strip=True)).replace("\u00a0", " ") for td in tds]
        if len(row) == len(header):
            data.append(row)

    if not data:
        raise RuntimeError("No data rows parsed. Table schema may have changed.")

    df_scrape = pd.DataFrame(data, columns=header)

    df_scrape = _infer_and_coerce_numeric(df_scrape, min_numeric_share=0.8)

except Exception as e:
    print("Scrape failed, using inline demo table:", e)
    html = '<table><tr><th>Country</th><th>Population</th></tr><tr><td>Demo</td><td>1,234,567</td></tr></table>'
    soup = BeautifulSoup(html, "html.parser")
    rows = [[c.get_text(strip=True) for c in tr.find_all(["th", "td"])] for tr in soup.find_all("tr")]
    header, *data = [r for r in rows if r]
    df_scrape = pd.DataFrame(data, columns=header)
    if "Population" in df_scrape.columns:
        df_scrape["Population"] = pd.to_numeric(df_scrape["Population"].str.replace(",", ""), errors="coerce")

v_scrape = validate(df_scrape, list(df_scrape.columns)); v_scrape
_ = save_csv(df_scrape, prefix="scrape", site="wikipedia", table="countries_population")


Saved /Users/murli/Desktop/bootcamp_mihir_subramaniam/homework/homework4/data/raw/scrape_site-wikipedia_table-countries_population_20250821-165726.csv


In [ ]:
_ = save_csv(df_scrape, prefix='scrape', site='example', table='markets')

## Documentation
# API Source:

- source: [alphavantage] https://www.alphavantage.co/query 
- Parameters:  
  - function=TIME_SERIES_DAILY  
  - symbol=MSFT 
  - outputsize=compact 
- Fallback: yfinance library (3-month daily adjusted close data for MSFT)  

# Scrape Source:
-  [Wikipedia, List of countries and dependencies by population] https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_population 
- Table Description: first wikitable containing country names, population estimates, and related metadata.  
- Parsing: using BeautifulSoup, cleaned headers, stripped whitespace and footnotes.  
- Validation: numeric vs. text columns inferred by heuristic (≥80% numeric → coerced to numeric).


# Assumptions & Risks

- API:
  - rate limits on Alpha Vantage (5 requests/min, 500/day).  
  - api key must remain private (.env file not committed).  
- Scrape:
  - wikipedia tables may change format (header names, footnotes, new columns).  
  - parsing assumes consistent structure of the first wikitable.  

# Data storage
- all processed files saved to data/raw